In [6]:
!pip install langchain_community  
!pip install pypdf

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
    --------------------------------------- 0.1/2.5 MB 812.7 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.5 MB 845.5 kB/s eta 0:00:03
   - -------------------------------------- 0.1/2.5 MB 722.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.5 MB 607.9 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.5 MB 731.4 kB/s eta 0:00:04
   ------ --------------------------------- 0.4/2.5 MB 1.3 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.5 MB 2.8 MB/s eta 0:00:01
   --------------------------- ------------ 1.7/2.5 MB 4.2 MB/s eta 0:00:01
   -------------------------------------- - 2.4/2.5 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   --------------------

In [ ]:
pip install -qU langchain-ollama

In [ ]:
!pip install langchain-groq

In [12]:
import os

from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

In [1]:
file_path='SM 980E-5 SN A50003-A50051--A50054-A50136--A50141 & UP (CEBM033006).pdf'  

In [2]:
from langchain_community.document_loaders import PyPDFLoader  # desde la libreria langchain_community voy a importar,  la clase PyPDFLoader   # una clase es un objeto que tiene atributos y metodos

loader = PyPDFLoader(file_path)  # se pasa la ruta del archivo el pdf, en este la ruta de arriba
pages = []     # se crea una lista vacia llamada pages
async for page in loader.alazy_load(): # para cada pagina en el loader (que es quien obtiene el cargador del pdf)
    pages.append(page)   # se agrega cada pagina , a la lista pages

In [3]:
len(pages) # cantidad de paginas

1395

In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3",
)

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

question = "what happends if water gets inside the connector?"

pages_new = [page.page_content for page in pages][116]
vectorstore = InMemoryVectorStore.from_texts(
    [pages_new],
    embedding=embeddings,
)


retriever = vectorstore.as_retriever()


retrieved_documents = retriever.invoke(question)


retrieved_documents[0].page_content

'10 Structure and functions CEN10067-01\n7\nNOTE: The pilot valve/hoist valve hydraulic circuit is\nrouted through the bleeddown manifold located on the\nleft frame rail. The hois t limit solenoid and pilot\noperated check valve described below are contained\nin the manifold. Refer to Structure, functions and\nmaintenance standard section Steering circuit  for\nmore information about the bleeddown manifold.\nHoist limit solenoid\nHoist limit solenoid (5, Figure 10-5), located in the\nbleeddown manifold, is used in the hydraulic circuit to\nprevent maximum hoist cylinder extension.\nThe 3-way valve solenoid valve is controlled by a\nproximity switch (hoist limit  switch) located near the\nbody pivot and the right rear suspension upper mount.\nWhen the body is nearly fully raised, the body\nactivates the magnetic proximity switch, signaling the\nsolenoid to open the “raise” pilot line to the tank, and\nclose the “raise” pilot line to the hoist valve, thus\nstopping further oil flow to th

In [6]:
retrieved_documents[0]

Document(id='4eef34d6-fd07-49eb-a14c-04b4a4a064fa', metadata={}, page_content='10 Structure and functions CEN10067-01\n7\nNOTE: The pilot valve/hoist valve hydraulic circuit is\nrouted through the bleeddown manifold located on the\nleft frame rail. The hois t limit solenoid and pilot\noperated check valve described below are contained\nin the manifold. Refer to Structure, functions and\nmaintenance standard section Steering circuit  for\nmore information about the bleeddown manifold.\nHoist limit solenoid\nHoist limit solenoid (5, Figure 10-5), located in the\nbleeddown manifold, is used in the hydraulic circuit to\nprevent maximum hoist cylinder extension.\nThe 3-way valve solenoid valve is controlled by a\nproximity switch (hoist limit  switch) located near the\nbody pivot and the right rear suspension upper mount.\nWhen the body is nearly fully raised, the body\nactivates the magnetic proximity switch, signaling the\nsolenoid to open the “raise” pilot line to the tank, and\nclose th

In [19]:

llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    groq_api_key="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"
)


In [20]:
TEMPLATE="""
Responde la pregunta {pregunta} en base a la información del
{texto}

El formato de salida debe ser el siguiente:{format_instructions}
"""

In [21]:
class ContentGenScript(BaseModel):
    respuesta: str = Field(description="Respuesta")


parser= JsonOutputParser(pydantic_object=ContentGenScript)

In [22]:
prompt = PromptTemplate(
    input_variables=["texto","pregunta"],
    template=TEMPLATE,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser) #objeto que concatena todo y permite ejecutar

texto = str(retrieved_documents)

pregunta = question

result = chain.invoke({"texto": texto, "pregunta":pregunta})

result['text']

{'respuesta': 'Water inside the connector can cause damage to the hydraulic system and potentially lead to a loss of control or failure of the hoist system.'}

In [18]:
print(texto)

[Document(id='4eef34d6-fd07-49eb-a14c-04b4a4a064fa', metadata={}, page_content='10 Structure and functions CEN10067-01\n7\nNOTE: The pilot valve/hoist valve hydraulic circuit is\nrouted through the bleeddown manifold located on the\nleft frame rail. The hois t limit solenoid and pilot\noperated check valve described below are contained\nin the manifold. Refer to Structure, functions and\nmaintenance standard section Steering circuit  for\nmore information about the bleeddown manifold.\nHoist limit solenoid\nHoist limit solenoid (5, Figure 10-5), located in the\nbleeddown manifold, is used in the hydraulic circuit to\nprevent maximum hoist cylinder extension.\nThe 3-way valve solenoid valve is controlled by a\nproximity switch (hoist limit  switch) located near the\nbody pivot and the right rear suspension upper mount.\nWhen the body is nearly fully raised, the body\nactivates the magnetic proximity switch, signaling the\nsolenoid to open the “raise” pilot line to the tank, and\nclose t